In [1]:
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.chdir('../..')

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import torch
from collections import OrderedDict
from pointcept.engines.defaults import (
    default_argument_parser,
    default_config_parser,
    default_setup,
)
from pointcept.datasets import build_dataset_iseg, collate_fn_iseg
from pointcept.models import build_model
import pointcept.utils.comm as comm

/data/ext_workspace/yuzijian2/.conda/envs/iseg3d_ptv3_2/lib/python3.8/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


cfg

In [3]:
args = default_argument_parser().parse_args('')
args.config_file = 'configs/scannet/iseg-agile3d-v1m1.py'
args.options = {'save_path': 'exp/test/models_ckpt'}
args

Namespace(config_file='configs/scannet/iseg-agile3d-v1m1.py', dist_url='auto', machine_rank=0, num_gpus=1, num_machines=1, options={'save_path': 'exp/test/models_ckpt'})

In [4]:
cfg = default_config_parser(args.config_file, args.options)
cfg = default_setup(cfg)
cfg

Config (path: configs/scannet/iseg-agile3d-v1m1.py): {'weight': None, 'resume': False, 'evaluate': True, 'test_only': False, 'seed': 31129887, 'save_path': 'exp/test/models_ckpt', 'num_worker': 12, 'batch_size': 12, 'batch_size_val': None, 'batch_size_test': None, 'epoch': 200, 'eval_epoch': 100, 'clip_grad': None, 'sync_bn': False, 'enable_amp': True, 'amp_dtype': 'float16', 'empty_cache': False, 'empty_cache_per_epoch': False, 'find_unused_parameters': False, 'enable_wandb': True, 'wandb_project': 'AGILE3D', 'wandb_key': None, 'mix_prob': 0.0, 'param_dicts': None, 'semantic_ignore_label': -1, 'instance_ignore_label': -1, 'semantic_background_label': (0, 1), 'hooks': [{'type': 'CheckpointLoader'}, {'type': 'IterationTimer', 'warmup_iter': 2}, {'type': 'InformationWriter'}, {'type': 'ISegEvaluator', 'semantic_ignore': -1, 'instance_ignore': -1, 'semantic_background': (0, 1)}, {'type': 'CheckpointSaver', 'save_freq': 10}, {'type': 'PreciseEvaluator', 'test_last': False}], 'train': {'typ

model

In [6]:
cfg.weight = 'exp/scannet/iseg-agile3d-v1m1-3/model/epoch_50.pth'
model = build_model(cfg.model)
checkpoint = torch.load(
                cfg.weight,
                map_location=lambda storage, loc: storage.cuda(),
                weights_only=False,
            )
weight = OrderedDict()
for key, value in checkpoint["state_dict"].items():
    if not key.startswith("module."):
        key = "module." + key  # xxx.xxx -> module.xxx.xxx
    if comm.get_world_size() == 1:
        key = key[7:]  # module.xxx.xxx -> xxx.xxx
    weight[key] = value
load_state_info = model.load_state_dict(
    weight, strict=True
)